### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42325902,89610,79085,51530,0,0,1127,5,2025-02-13 22:00,2025-02-13 22:00,0,0,4332,3,1
34,42325442,73876,79085,3,0,0,1127,6,2025-02-14 19:14,2025-02-14 19:14,3,0,9807,1,1
35,42326828,90579,79085,51530,0,0,1128,0,2025-02-15 22:00,2025-02-15 22:00,1,1,3495,5,1
36,42084905,79085,47841,1349,17,69,1128,1,2025-02-16 21:58,2025-02-16 21:59,0,1,17063,5,1
37,42326827,79085,6349,51530,0,0,1128,2,2025-02-17 22:00,2025-02-17 22:00,1,6,2484,4,1
38,42329739,79085,90579,51530,0,0,1128,3,2025-02-18 22:00,2025-02-18 22:00,2,2,2913,4,1
39,42084903,48541,79085,1349,18,69,1128,4,2025-02-19 21:59,2025-02-19 21:59,1,4,14251,7,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,89610,20971,40503,2,16,1,0,0,Base Forte,10,10,11,42325902
1,79085,49271,29739,0,1,3,1,0,5-4-1a,8,9,10,42325902
0,73876,24106,39323,4,16,0,0,0,Pilfer 433_Mirr,10,8,9,42325442
1,79085,45854,30637,0,0,3,1,0,5-4-1a,5,7,6,42325442
0,90579,29423,34677,3,8,3,0,0,5 - 4,10,11,11,42326828
1,79085,40528,35274,0,2,1,0,0,5-4-1aSR,9,9,10,42326828
0,79085,43061,35575,1,1,2,1,0,5-4-1aSR,9,9,10,42084905
1,47841,26889,34375,1,11,4,1,0,copa2,11,11,10,42084905
0,79085,48842,31762,1,6,2,0,0,3-4-3BC,9,9,10,42326827
1,6349,21125,38205,4,20,0,0,0,P,12,9,10,42326827


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1574

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,47841,18,51,17,0,1,85,12,51573085017010,1349
1,6295,18,41,13,2,3,88,30,41558088013009,1349
2,23755,18,39,12,3,3,87,25,39562087012011,1349
3,79866,18,38,12,2,4,49,18,38531049012005,1349
4,78671,18,35,11,2,5,65,23,35542065011007,1349
5,79085,18,26,7,5,6,42,25,26517042007006,1349
6,89569,18,25,8,1,9,36,43,25493036008002,1349
7,122169,18,23,7,2,9,77,63,23514077007004,1349
8,124139,18,22,7,1,10,48,55,22493048007008,1349
9,48541,18,9,3,0,15,19,109,9410019003001,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()